In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install -q langchain langchain_community sentence_transformers modelscope torch datasets peft trl bitsandbytes accelerate transformers chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.7/40.7 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 39.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 59.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 70.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.9/310.9 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 617.9/617.9 kB 33.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 84.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 24.5 MB/s eta 

In [ ]:
import random
import re
import gc
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter,CharacterTextSplitter
from langchain_community.document_loaders import UnstructuredFileLoader
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate
from langchain_community.chat_models import ChatOllama
import pandas as pd
from langchain.schema import Document
from sentence_transformers import SentenceTransformer
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
)
import torch
from datasets import Dataset, load_dataset
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
from transformers import AutoProcessor, AutoModelForPreTraining, BitsAndBytesConfig, AutoTokenizer
import torch

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [ ]:
from transformers import AutoProcessor, AutoModelForPreTraining, BitsAndBytesConfig, AutoTokenizer
import torch
def split_dialogue_Q_A(conversation):
    pattern = re.compile(r'Q: (.*?)\nA: (.*?)\n', re.DOTALL)
    matches = pattern.findall(conversation)
    questionlist=[]
    answerlist=[]

    for match in matches:
        question, answer = match

        # if len(question.split(' '))>50 or len(answer.split(' '))>50 or len(question.split(' '))<3:
        #     continue
        questionlist.append(question)
        answerlist.append(answer)

        # print("Question:", question)
        # print("Answer:", answer)
        # print("-----")
        # exit(1)

    return questionlist,answerlist

In [ ]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

MODEL_NAME = "bigscience/bloom-7b1"

DEFAULT_SYSTEM_PROMPT = """
You are physicist Richard Feynman, well-known for your significant contributions to particle physics, quantum electrodynamics, and quantum mechanics. You have a ability for making difficult ideas understandable to others without diminishing their depth. You are known for your insatiable curiosity, sense of humor, and unorthodox teaching style. You enjoy solving problems and have a talent for telling stories.
""".strip()


In [ ]:
model_name = "moka-ai/m3e-base"

embeddings = HuggingFaceEmbeddings(model_name=model_name)
embeddings.client = SentenceTransformer(
        embeddings.model_name, device=DEVICE)

<ipython-input-6-75405089cb32>:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=model_name)
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/26.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/932 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/439k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
ragdb = Chroma(persist_directory="/content/drive/MyDrive/DATA298B/ChormaDBs/feynman", embedding_function=embeddings)

retriever = ragdb.as_retriever(search_kwargs={'k': 3})

<ipython-input-7-1e142866e725>:1: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  ragdb = Chroma(persist_directory="/content/drive/MyDrive/DATA298B/ChormaDBs/feynman", embedding_function=embeddings)


In [ ]:
def custom_format(docs):
  output = ""
  for doc in docs:
    doc = doc.page_content.split("A:")[-1]
    output += doc
    output += "\n"
  return output

def generate_training_prompt(question, response, system_prompt = DEFAULT_SYSTEM_PROMPT):
    tempt =f"""
    ### Instruction: {system_prompt}
    {custom_format(retriever.invoke(question))}
    ### Question:
    {question.strip()}
    ### Response:[/INST]
    {response}
    """.strip()
    return tempt

def create_model_and_processor(MODEL_NAME):

    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
    )


    model = AutoModelForPreTraining.from_pretrained(
        MODEL_NAME,
        use_safetensors=True,
        quantization_config=bnb_config,
        trust_remote_code=True,
        device_map="auto",
    )


    processor = AutoProcessor.from_pretrained(MODEL_NAME)


    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"

    return model, processor, tokenizer


MODEL_NAME = "bigscience/bloom-7b1"
model, processor, tokenizer = create_model_and_processor(MODEL_NAME)


config.json:   0%|          | 0.00/739 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/28.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.16G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/222 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

In [ ]:
file_path = '/content/drive/MyDrive/DATA298B/data/interview_transcript_Q&A.txt'


with open(file_path, 'r', encoding='utf-8') as file:
    texts = file.read()
questionlist,answerlist = split_dialogue_Q_A(texts)

print(len(questionlist))

218


In [ ]:
tmpList = []
trainList = []
testList = []
for j,(question,answer) in enumerate(zip(questionlist,answerlist)):
  if type(question) != str:
    continue
  tmp_dict = {"question":question, "answer":answer, "training_prompt":generate_training_prompt(question,answer,system_prompt = DEFAULT_SYSTEM_PROMPT)}
  # print(generate_training_prompt(question,answer,system_prompt = DEFAULT_SYSTEM_PROMPT))
  # exit(1)
  tmpList.append(tmp_dict)
  # if j == 200:
  #   break
  if j % 100 == 0:
    print(j)
  if random.uniform(0,1) < .2:
    testList.append(tmp_dict)
  else:
    trainList.append(tmp_dict)

data = Dataset.from_list(tmpList)
train_data = Dataset.from_list(trainList)
test_data = Dataset.from_list(testList)

0
100
200


In [ ]:
from peft import LoraConfig, TaskType
lora_r = 32
lora_alpha = 16
lora_dropout = 0.1
lora_target_modules = [
    "q_proj",
    "up_proj",
    "o_proj",
    "k_proj",
    "down_proj",
    "gate_proj",
    "v_proj",
]

peft_config = LoraConfig(task_type=TaskType.CAUSAL_LM, inference_mode=False, r=32, lora_alpha=16, lora_dropout=0.1)

'''peft_config = LoraConfig(
    r=lora_r,
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    target_modules=lora_target_modules,
    bias="none",
    task_type="CAUSAL_LM",
)'''

'peft_config = LoraConfig(\n    r=lora_r,\n    lora_alpha=lora_alpha,\n    lora_dropout=lora_dropout,\n    target_modules=lora_target_modules,\n    bias="none",\n    task_type="CAUSAL_LM",\n)'

In [ ]:
import os

In [ ]:
# Define output directory for saving checkpoints
OUTPUT_DIR = "/content/drive/MyDrive/DATA298B/trained_model_feynman/5epoch/"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Training arguments
training_arguments = TrainingArguments(
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    optim="paged_adamw_32bit",
    logging_steps=5,
    learning_rate=5e-4,
    fp16=True,
    max_grad_norm=0.3,
    num_train_epochs=5,
    eval_strategy="epoch",  # Evaluate at the end of each epoch
    eval_steps = 10,
    save_strategy="epoch",  # Save after every epoch
    save_total_limit=2,  # Keep only the latest 5 checkpoints
    warmup_ratio=0.05,
    group_by_length=True,
    output_dir=OUTPUT_DIR,
    report_to="none",
    save_safetensors=True,
    lr_scheduler_type="cosine",
    seed=42,
    load_best_model_at_end=True,  # Optional: Load best model at the end
)

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=test_data,
    peft_config=peft_config,
    dataset_text_field="training_prompt",
    max_seq_length=150,
    tokenizer=tokenizer,
    args=training_arguments,
)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:300: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:328: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/182 [00:00<?, ? examples/s]

Map:   0%|          | 0/36 [00:00<?, ? examples/s]

In [ ]:
# Train the model and save checkpoints after each epoch
trainer.train()

# Save the final model at the end of training
trainer.save_model(OUTPUT_DIR)

Epoch,Training Loss,Validation Loss
0,2.347300,1.530512
1,1.361300,1.215088
2,1.217700,1.144706
3,1.114100,1.124318
4,1.144600,1.120862


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


In [ ]:
trainer2 = SFTTrainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=test_data,
    peft_config=peft_config,
    dataset_text_field="training_prompt",
    max_seq_length=150,
    tokenizer=tokenizer,
    args=training_arguments,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:300: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:328: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/182 [00:00<?, ? examples/s]

Map:   0%|          | 0/36 [00:00<?, ? examples/s]

In [ ]:
trainer2.train(resume_from_checkpoint = True)

/usr/local/lib/python3.10/dist-packages/transformers/trainer.py:3354: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(os.path.join(checkpoint, OPTIMIZER_NAME), map_

Epoch,Training Loss,Validation Loss


TrainOutput(global_step=55, training_loss=0.0, metrics={'train_runtime': 0.0772, 'train_samples_per_second': 11789.941, 'train_steps_per_second': 712.579, 'total_flos': 4776466952159232.0, 'train_loss': 0.0, 'epoch': 4.835164835164835})

In [ ]:
trainer.model.save_pretrained("feynman_finetune_5e4")

In [ ]:
finetuned_model = PeftModel.from_pretrained(
    model,
    "feynman_finetune_5e4",
    is_trainable=True
)
finetuned_model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): BloomForCausalLM(
      (transformer): BloomModel(
        (word_embeddings): Embedding(250880, 4096)
        (word_embeddings_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (h): ModuleList(
          (0-29): 30 x BloomBlock(
            (input_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
            (self_attention): BloomAttention(
              (query_key_value): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=12288, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=32, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=32, out_features=12288, bias=False)
                )
            

In [ ]:
import shutil
from google.colab import files

files_to_download = [
    "feynman_finetune_5e4/README.md",
    "feynman_finetune_5e4/adapter_config.json",
    "feynman_finetune_5e4/adapter_model.safetensors"
]

shutil.make_archive("feynman_finetune_5e4", 'zip', "feynman_finetune_5e4")

files.download("feynman_finetune_5e4.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>